# 导入模块

In [ ]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [ ]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [ ]:
# host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com"
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [ ]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
, om.order_type, om.union_rent_tag, tojo.app_type, tp.classify_id, tp.id 商品id, tpmn.name 机型
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id

-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
-- 商品表
left join db_digua_business.t_product tp on tp.id=tod.product_id
-- 商品型号
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )               -- 近15天数据
-- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-07-01'
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-07-15'
-- and hour(om.create_time) < 16
'''

df_order = query(sql1)
df_order.shape

In [ ]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected

from db_credit.risk
'''
df_risk = query(sql_risk)
# df_risk.loc[:, 'time_r'] = pd.to_datetime(df_risk.time.dt.date,errors="coerce")
# df_risk = df_risk.sort_values(by='time', ascending=False).groupby('id_card_r').head(1)

In [ ]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
# df_re = df_re.sort_values(by='time', ascending=False).groupby('id_card_re').head(1)

In [ ]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

In [ ]:
# sql_name = '''
# SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
# FROM db_digua_business.t_user_verify_order_record tuvor
# left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
# where tuvor.del_flag = 0 ORDER BY tuvor.update_time
# '''
# df_name = query(sql_name)

In [ ]:

# sql_stages = '''
# select 
# tos.order_id, tos.sort, tos.refund_date, tos.reality_refund_date
# from db_digua_business.t_order_stages tos
# where tos.sort = 2
# '''
# df_stages = query(sql_stages)

In [ ]:
sql_wl = '''
select
date_format(om.create_time, '%Y-%m-%d') 下单日期, tort.order_num order_number, tort.task_status
from db_digua_business.t_order_retention_task tort
left join db_digua_business.t_order om on om.id=tort.order_id
'''
df_wl = query(sql_wl)
df_wl.shape

# 提取订单风控数据

# 合并数据

In [ ]:
df_order.loc[:,'商品ID_机型'] = df_order.商品id.astype(str)+'_'+df_order.机型
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

# 处理日期

In [ ]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
df['hour'] = df['create_time'].dt.hour
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

# 渠道补充

In [ ]:
# qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)
# qudao_name_df.rename(columns={'订单号': '订单号1'}, inplace=True)
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号1',how = 'left')
# df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])

# 处理备注信息

In [ ]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  


def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()


df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

# 渠道归属

In [ ]:

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

# 订单去重

In [ ]:
# 保留不去重的数据
df_no_drop = df.copy()
# 订单去重
df = clean.order_drop_duplicates(df)

# 定义状态

In [ ]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
# , 'union_rent_tag', 'union_rent_rejected'
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
# note 2025-02-24修改
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

In [ ]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1) 

# 保留商家的数据

In [ ]:
df_contain = df.copy()

# 剔除商家数据

In [ ]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)

# 去重订单明细导出

In [ ]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

# 导入服务订单

In [ ]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

df_j = df[df["进件"]=="进件"]
df_j.shape

In [ ]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)

df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)


df['是否出库'] = np.where((df['人审拒绝']==0)&(df['客户取消']==0)&(df['无法联系']==0)&(df['待审核']==0)&(df['是否出库']==1), 1, 0)
df["出库前风控强拒"] = np.where((df["审核状态"]=='出库前风控强拒')&(df['是否出库']==0),1,0)
df["机审强拒"] = np.where((df["审核状态"]=='机审强拒')&(df['是否出库']==0),1,0)
df['机审通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0), 1, 0)
df['风控通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0)&(df['人审拒绝']==0), 1, 0)

df['已退款'] = np.where((df['风控通过件']==1)&(df['审核状态']=='已退款'), 1, 0)

df['是否二手'] = np.where(df['product_name'].str.contains(r'99新|95新|准新|90新'), 1, 0)

df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)

# 导入台账数据

In [ ]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023")  

f_path_ck = "D:/excel_data/台账/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025") 
df_ck.shape

In [ ]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape

In [ ]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
base_date = pd.Timestamp('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')


# 每日各渠道出库单数

In [ ]:
# 按转化日期看每日各渠道出库单数
df_weekday_zh = pd.crosstab(dfck["日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数
df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数（纯租物）
df_ly_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
# 按下单日期看每日支付宝直播商品出库单数
df_zfb_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 按下单日期看 免押 出库单数
df_my = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)

# 总体转化

## 剔除拒量订单

In [ ]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]
day_name_cn_mapping = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期日'
}

In [ ]:
# def year_sum(df, year):
#     df2 = df[df.年份==year]
#     df2_ck = df2[df2.是否出库==1]
#     df2_group = df2_ck.groupby('下单月份').agg({'是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum'}).rename(columns={'是否出库': '出库数', 'purchase_amount': '采购金额', 'new_actual_money': '签约价'})
#     return df2_group
# df_group_2023 = year_sum(df, 2023)
# df_group_2024 = year_sum(df, 2024)
# df_group_2025 = year_sum(df, 2025)
# df_group_2025
# with pd.ExcelWriter(f'F:/需求/七月需求/23-25年出库数据1.xlsx', engine='xlsxwriter') as writer:
#     df_group_2023.to_excel(writer, sheet_name='2023年出库数据')
#     df_group_2024.to_excel(writer, sheet_name='2024年出库数据')
#     df_group_2025.to_excel(writer, sheet_name='2025年出库数据')

# df_new = df[~df.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机'])]
# df_new_g = df_new[df_new.是否出库==1].groupby('下单月份').agg({'是否出库': 'sum', 'new_actual_money': 'sum'}).rename(columns={'new_actual_money': '签约价', '是否出库': '出库'})
# df_g = df.groupby('下单月份').agg({'是否进件': 'sum'}).rename(columns={'是否进件': '进件'})
# df_group = df_g.merge(df_new_g, on='下单月份', how='left').reset_index()
# df_group[['进件', '出库', '签约价']].set_index('进件')

In [ ]:
# 总体数据
# df = df[df.归属渠道!='京东渠道']
# df2 = df2[df2.归属渠道!='京东渠道'] 
df_all = all_models.data_group(df, df2, df_risk_examine, '下单日期')
df_all['出库（按下单时间）'] = df_all['出库']
df_all['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
df_all = df_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_all_new = df_all.reset_index()
df_all_new.insert(0, '星期', df_all_new['下单日期'].apply(lambda x: day_name_cn_mapping[x.day_name()]))
df_all_new.insert(1, '月份', df_all_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_all_new['下单日期'].astype(str).str.split('-').str[1])

In [ ]:
df_all_new[df_all_new['下单日期'] == '2025-07-15']

In [ ]:
# df_all.to_excel(f'F:/需求/瑞彤需求/支付宝数据_{t_date}.xlsx')

# 搜索渠道转化

In [ ]:
df_ss = df[df['归属渠道']=='搜索渠道']
df_ss2 = df2[df2['归属渠道']=='搜索渠道']
df_ss_group = all_models.data_group(df_ss, df_ss2, df_risk_examine, '下单日期')

# df_ss_group['出库（按下单时间）'] = df_weekday['搜索渠道'][:-1]
df_ss_group['出库（按下单时间）'] = df_ss_group['出库']
df_ss_group['出库（按转化时间）'] = df_weekday_zh['搜索渠道'][:-1]
df_ss_group = df_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_ss_group_new = df_ss_group.reset_index()
# df_ss_group

# 单人聊天 转化

In [ ]:
df_dr = df[df['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
df_dr2 = df2[df2['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
df_dr_group = all_models.data_group(df_dr, df_dr2, df_risk_examine,  '下单日期')

# df_dr_group['出库（按下单时间）'] = df_weekday['单人聊天会话中的小程序消息卡片（分享）'][:-1]
df_dr_group['出库（按下单时间）'] = df_dr_group['出库']
try:
    df_dr_group['出库（按转化时间）'] = df_weekday_zh['单人聊天会话中的小程序消息卡片（分享）'][:-1]
except:
    df_dr_group['出库（按转化时间）'] = 0
df_dr_group = df_dr_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']].fillna(0)
df_dr_group_new = df_dr_group.reset_index()

# 芝麻渠道转化

In [ ]:
df_zm = df[df['归属渠道']=='芝麻租物']
df_zm2 = df2[df2['归属渠道']=='芝麻租物']
df_zm_group = all_models.data_group(df_zm, df_zm2, df_risk_examine, '下单日期')

# df_zm_group['出库（按下单时间）'] = df_weekday['芝麻租物'][:-1]
df_zm_group['出库（按下单时间）'] = df_zm_group['出库']
df_zm_group['出库（按转化时间）'] = df_weekday_zh['芝麻租物'][:-1]
df_zm_group = df_zm_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_zm_group_new = df_zm_group.reset_index()
# df_zm_group

# 纯租物 转化

In [ ]:
#逻辑是否需要优化   
df_zw = df[df['来源渠道']=='芝麻信用']
df_zw2 = df2[df2['来源渠道']=='芝麻信用']
df_zw_group = all_models.data_group(df_zw, df_zw2, df_risk_examine, '下单日期')

# df_zw_group['出库（按下单时间）'] = df_ly_ck['芝麻租物'][:-1]
df_zw_group['出库（按下单时间）'] = df_zw_group['出库']
df_zw_group = df_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','进件出库率','订单出库率']]
df_zw_group_new = df_zw_group.reset_index()

# 抖音转化

In [ ]:
df_dy = df[df['归属渠道']=='抖音渠道']
df_dy2 = df2[df2['归属渠道']=='抖音渠道']
# df_dy_group = all_models.data_group(df_dy, df_dy2, df_risk_examine, '下单日期')
# df_dy_group = clean.continuous_dates(df_dy_group)
# df_dy_group['出库（按下单时间）'] = df_dy_group['出库']
# try:
#     df_dy_group['出库（按转化时间）'] = df_weekday_zh['抖音渠道'][:-1]
# except:
#     df_dy_group['出库（按转化时间）'] = 0
# df_dy_group = df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']].fillna(0) 
# df_dy_group_new = df_dy_group.reset_index()


In [ ]:
classify_id_list = [125,126,127,185]
dfck.loc[:, '抖音电脑'] = np.where((dfck.归属渠道=='抖音渠道')&(dfck.classify_id.isin(classify_id_list)), 1, 0)
# 按转化日期看抖音电脑品类出库
dfck_dy_pc = pd.crosstab(index=dfck['日期'], columns=dfck['抖音电脑'],margins=True)
# 电脑
df_dy_classify = df_dy[df_dy.classify_id.isin(classify_id_list)]
df_d2y_classify = df_dy2[df_dy2.classify_id.isin(classify_id_list)]
df_dy_group = all_models.data_group(df_dy_classify, df_d2y_classify, df_risk_examine, '下单日期')
df_dy_group = clean.continuous_dates(df_dy_group)
df_dy_group['出库（按下单时间）'] = df_dy_group['出库']
try:
    df_dy_group['出库（按转化时间）'] = dfck_dy_pc[1][:-1]
except:
    df_dy_group['出库（按转化时间）'] = 0
df_dy_group = df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒",'出库（按下单时间）','出库（按转化时间）', '进件出库率','订单出库率']].fillna(0) 
df_dy_group_new = df_dy_group.reset_index()

# 支付宝直播转化

In [ ]:
# # 支付宝直播
df_zfb = df[df['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb2 = df2[df2['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb_group = all_models.data_group(df_zfb, df_zfb2, df_risk_examine, '下单日期')
df_zfb_group = clean.continuous_dates(df_zfb_group)

# df_zfb_group['出库（按下单时间）'] = df_zfb_ck['支付宝直播商品'][:-1]
df_zfb_group['出库（按下单时间）'] = df_zfb_group['出库']
df_zfb_group['出库（按转化时间）'] = df_zfb_group['出库']
df_zfb_group = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）', '出库（按转化时间）','进件出库率','订单出库率']]
df_zfb_group1 = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','订单出库率']].reset_index(names=['下单日期'])
# df_zfb_group

In [ ]:
df1_ck = df_zfb[df_zfb['是否出库']==1]
Today_zfb = datetime.now().strftime('%Y%m%d%H')

# 总体剔除直播数据

In [ ]:
df_tc = df_all[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dr_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dy_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_zfb_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]
df_tc.fillna(df_all[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dr_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_zfb_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']], inplace=True)
df_tc['预授权通过率'] = df_tc['进件数'] / df_tc['去重订单数']
df_tc['预授权通过率'] = df_tc['预授权通过率'].apply(lambda x: format(x, '.2%'))
df_tc['强拒比例'] = df_tc['机审强拒'] / df_tc['进件数']
df_tc['强拒比例'] = df_tc['强拒比例'].apply(lambda x: format(x, '.2%'))
df_tc['进件出库率'] = df_tc['出库（按下单时间）'] / df_tc['进件数']
df_tc['进件出库率'] = df_tc['进件出库率'].apply(lambda x: format(x, '.2%'))
df_tc['订单出库率'] = df_tc['出库（按下单时间）'] / df_tc['去重订单数']
df_tc['订单出库率'] = df_tc['订单出库率'].apply(lambda x: format(x, '.2%'))
df_tc = df_tc[['去重订单数', '进件数', '预授权通过率', '机审强拒', '强拒比例', '出库（按下单时间）', '出库（按转化时间）', '进件出库率', '订单出库率']].reset_index()
# df_tc

# 京享租 2025-05-20正式上线

In [ ]:
df_no_drop_jd = df_no_drop[df_no_drop.归属渠道=='京东渠道']
df_no_drop_jd = df_no_drop_jd[df_no_drop_jd.下单日期>='2025-05-20']
df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件=='进件', 1, 0)
df_no_drop_jd_g = df_no_drop_jd.groupby('下单日期').agg({'order_id': 'count', '创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
df_no_drop_jdsc_g = df_no_drop_jd[df_no_drop_jd.app_type=='JD_APP'].groupby('下单日期').agg({'order_id': 'count'}).rename(columns={'order_id': '创建订单数'})

In [ ]:
df_jd = df[df['归属渠道']=='京东渠道']
df_jd2 = df2[df2['归属渠道']=='京东渠道']
df_jd = df_jd[df_jd.下单日期>='2025-05-20']
df_jd2 = df_jd2[df_jd2.下单日期>='2025-05-20']
df_jd_group = all_models.data_group(df_jd, df_jd2, df_risk_examine, '下单日期')


df_jd_group['出库（按下单时间）'] = df_jd_group['出库']
try:
    df_jd_group['出库（按转化时间）'] = df_weekday_zh['京东渠道'][:-1]
except:
    df_jd_group['出库（按转化时间）'] = 0
df_jd_group = df_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jd_group_new = df_jd_group.reset_index()
df_jd_group_new = df_no_drop_jd_g.merge(df_jd_group_new, on='下单日期', how='inner')

## 京东商城

In [ ]:
df_jdsc = df_jd[df_jd.app_type=='JD_APP']
df_jdsc2 = df_jd2[df_jd2.app_type=='JD_APP']
df_jdsc_group = all_models.data_group(df_jdsc, df_jdsc2, df_risk_examine, '下单日期')
df_jdsc_group['出库（按下单时间）'] = df_jdsc_group['出库']

df_jdsc_group = df_jdsc_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jdsc_group_new = df_jdsc_group.reset_index()
df_jdsc_group_new = df_no_drop_jdsc_g.merge(df_jdsc_group_new, on='下单日期', how='left')


# 渠道退货订单数

In [ ]:
# 输出df归属渠道的种类
# df['归属渠道'].unique()
# 获取df中所以退货拒签的订单
df_qudao = df[df['是否出库']==1]

# 将归属渠道中的渠道数据依次取出芝麻租物， 搜索渠道，单人会话，S量，抖音渠道，，京东渠道（且前面六个渠道都需排除拒量），拒量(已存在是否拒量)的退货订单数
df_qudao['芝麻租物'] = np.where((df_qudao.归属渠道=='芝麻租物')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['搜索渠道'] = np.where((df_qudao.归属渠道=='搜索渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['单人会话'] = np.where((df_qudao.归属渠道=='单人会话')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
conditions_s = ['八派信息', 'CPS直播', '派金花', '九州信息', '勉丫租', '哈银', '租瓜直播', '美仑美奂',
                        '分期乐1', '分期乐2', '98租超', '推一推', '硬派抖音']
df_qudao['S量'] = np.where((df_qudao.归属渠道.isin(conditions_s))&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['抖音渠道'] = np.where((df_qudao.归属渠道=='抖音渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)

df_qudao['拒量'] = np.where((df_qudao['是否拒量']==1)&(df_qudao.退货拒签==1), 1, 0)

df_qudao['京东渠道'] = np.where((df_qudao.归属渠道=='京东渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
# 将各个渠道的订单数相加
df_qudao_group = df_qudao.groupby('下单日期').agg({'order_number':'count','退货拒签':'sum','芝麻租物': 'sum', '搜索渠道': 'sum', '单人会话': 'sum', 'S量': 'sum', '抖音渠道': 'sum', '拒量': 'sum', '京东渠道': 'sum'}).rename(columns={'order_number': '总体出库订单数', '退货拒签': '拒收订单数'}).reset_index()


df_qudao_group

# 免审转化

判断免人审的单可以用t_order_risk表中的risk json串中的is_vip=1和status=0

In [ ]:
df_ms = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_ms_group = all_models.data_group(df_ms, df_ms2, df_risk_examine,  '下单日期')

df_ms_group = df_ms_group[["进件数","人审拒绝", "客户取消","出库","待审核","进件出库率","取消率","人审拒绝率"]] 
# df_ms_group = df_ms_group[["进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    # "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
df_ms_group_new = df_ms_group.reset_index()
# df_ms_group

In [ ]:
# df_241205 = df_ms[df_ms.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略', regex=True)==True]
# df_241205.loc[:, '出库前强拒_拒量'] = np.where(((df_241205.qvt_risk == '0')|(df_241205.qvt_risk.isna()))&(df_241205.qvt_result == '1'), 1, 0)
# df_241205_new = df_241205[df_241205.出库前强拒_拒量==1][['下单日期', 'order_number']]
# df_ms2_new = df_ms2[df_ms2.出库前风控强拒==1][['下单日期', 'order_number']]
# pd.concat([df_ms2_new, df_241205_new]).to_excel('F:/需求/总需求/免审出库前强拒.xlsx', index=False)

In [ ]:
# df_ms_group[["机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].to_excel('F:/需求/总需求/免审转化.xlsx')


In [ ]:
# df_fms2 = df2[~((df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0'))&(df2.机审通过件==1)]
# df_fms2[df_fms2.下单日期=='2025-02-24']['order_number'].to_excel('F:/需求/总需求/非免审转化订单.xlsx', index=False)

## 各渠道每日免审出库单数统计

In [ ]:
# 剔除据量的数据
# dfck_ms = dfck[(~dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True)&(dfck.status!=12)]
dfck_ms = clean.drop_rejected_merchant(dfck)
dfdfck=dfck_ms[(dfck_ms.is_vip=='1')&(dfck_ms.status_result=='0')]

dfdfck_ck = pd.crosstab(dfdfck["下单日期"],dfdfck["归属渠道"],margins=True)[['芝麻租物','搜索渠道']][:-1]

# 删除原先索引
# dfdfck_ck = dfdfck_ck.reset_index(drop=True)
dfdfck_ck.rename(columns={'芝麻租物': '芝麻租物免审出库', '搜索渠道': '搜索免审出库'}, inplace=True)
dfdfck_ck = clean.continuous_dates(dfdfck_ck)
dfdfck_ck = dfdfck_ck.reset_index(names=['下单日期'])[['芝麻租物免审出库', '搜索免审出库']]#.set_index('下单日期')


## 租物  搜索的免审进件

In [ ]:
dfms = df[(df.is_vip=='1')&(df.status_result=='0')]
dfms_jj = pd.crosstab(dfms["下单日期"],dfms["归属渠道"])
# 删除原先索引
dfms_jj = dfms_jj.reset_index(drop=True)
# 添加下单日期
dfms_jj['下单日期'] = df_ms_group_new['下单日期']
dfms_jj = dfms_jj[['下单日期', "芝麻租物","搜索渠道"]]
dfms_jj.rename(columns={'芝麻租物': '芝麻租物免审进件', '搜索渠道': '搜索渠道免审进件'}, inplace=True)
# dfms_jj

In [ ]:
# 整合
df_ms_new = dfms_jj.copy()
df_ms_new.insert(0, '月份', df_ms_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_ms_new['下单日期'].astype(str).str.split('-').str[1])
df_ms_new.insert(2, '总体进件', df_all_new['进件数'])
df_ms_new.insert(3, '芝麻租物进件', df_zm_group_new['进件数'])
df_ms_new.insert(4, '免审进件', df_ms_group_new['进件数'])
df_ms_new.loc[:, '免审进件占比'] = df_ms_new['免审进件'] / df_ms_new['总体进件']
df_ms_new.loc[:, '免审进件占比'] = df_ms_new['免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '芝麻租物免审进件占比'] = df_ms_new['芝麻租物免审进件'] / df_ms_new['芝麻租物进件']
df_ms_new.loc[:, '芝麻租物免审进件占比'] = df_ms_new['芝麻租物免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '搜索免审进件占比'] = df_ms_new['搜索渠道免审进件'] / df_ss_group_new['进件数']
df_ms_new.loc[:, '搜索免审进件占比'] = df_ms_new['搜索免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '总体出库'] = df_all_new['出库（按下单时间）']
df_ms_new.loc[:, '芝麻租物出库'] = df_zm_group_new['出库（按下单时间）']
df_ms_new.loc[:, '免审出库'] = df_ms_group_new['出库']
df_ms_new2 = pd.concat([df_ms_new, dfdfck_ck], axis=1)
# df_ms_new2
df_ms_new2.loc[:, '免审转化率'] = df_ms_new2['免审出库'] / df_ms_new2['免审进件']
df_ms_new2.loc[:, '免审转化率'] = df_ms_new2['免审转化率'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '免审出库占比'] = df_ms_new2['免审出库'] / df_ms_new2['总体出库']
df_ms_new2.loc[:, '免审出库占比'] = df_ms_new2['免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '芝麻租物免审出库占比'] = df_ms_new2['芝麻租物免审出库'] / df_ms_new2['芝麻租物出库']
df_ms_new2.loc[:, '芝麻租物免审出库占比'] = df_ms_new2['芝麻租物免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '搜索免审出库占比'] = df_ms_new2['搜索免审出库'] / df_ss_group_new['出库（按下单时间）']
df_ms_new2.loc[:, '搜索免审出库占比'] = df_ms_new2['搜索免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '机审通过'] = df_all_new['机审通过件']
df_ms_new2.loc[:, '非免审进件'] = df_ms_new2['机审通过'] - df_ms_new2['免审进件']
df_ms_new2.loc[:, '非免审出库'] = df_ms_new2['总体出库'] - df_ms_new2['免审出库']
df_ms_new2.loc[:, '非免审转化率'] = df_ms_new2['非免审出库'] / df_ms_new2['非免审进件']
df_ms_new2.loc[:, '非免审转化率'] = df_ms_new2['非免审转化率'].apply(lambda x: format(x, '.2%'))
df_ms_new3 = pd.concat([df_ms_new2, df_ms_group_new], axis=1)
# df_ms_new3.set_index('月份')

## 免审订单转化

In [ ]:
# 免审
df_ms1_o = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2_o = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_ms_o_group = all_models.data_group(df_ms1_o, df_ms2_o, df_risk_examine, '下单日期')
df_ms_o_group = df_ms_o_group[["进件数","人审拒绝","客户取消","出库","待审核","进件出库率","取消率","人审拒绝率",'出库前风控强拒','出库前强拒比例','无法联系','无法联系占比']]
df_ms_o_group_new = df_ms_o_group.reset_index()
df_ms_o_group_new.insert(0, '月份', df_ms_o_group_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_ms_o_group_new['下单日期'].astype(str).str.split('-').str[1])
# df_ms_o_group_new.set_index('月份')

#  免押订单转化率 全免押

In [ ]:
df_qmy = df[df['押金类型']=='全免押']
df_qmy2 = df2[df2['押金类型']=='全免押']
df_qmy_group = all_models.data_group(df_qmy, df_qmy2, df_risk_examine, '下单日期')
df_qmy_group['出库（按下单时间）'] = df_my['全免押'][:-1]
df_qmy_group = df_qmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","待审核", '出库（按下单时间）', '进件出库率', '取消率', '人审转化率', '人审拒绝率']].fillna(0)
df_qmy_group_new = df_qmy_group.reset_index()
df_qmy_group_new.insert(1, '总进件', df_all_new['进件数'])
df_qmy_group_new.insert(2, '免押进件占比', (df_qmy_group_new.进件数/df_qmy_group_new.总进件).map(lambda x: format(x, '.2%')))
# df_qmy_group_new.set_index('下单日期')

# 非免押（部分免押）

In [ ]:
df_fmy = df[df['押金类型']=='部分免押']
df_fmy2 = df2[df2['押金类型']=='部分免押']
df_fmy_group = all_models.data_group(df_fmy, df_fmy2, df_risk_examine, '下单日期')
df_fmy_group['待审核'] = 0
df_fmy_group = df_fmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消", '待审核']]
try:
    df_fmy_group.loc[:, '出库'] = df_my['部分免押'][:-1]
except:
    df_fmy_group.loc[:, '出库'] = 0
df_fmy_group_new = df_fmy_group.reset_index()
df_fmy_group_new.insert(1, '总进件', df_all_new['进件数'])
df_fmy_group_new.insert(2, '免押进件占比', (df_fmy_group_new.进件数/df_fmy_group_new.总进件).map(lambda x: format(x, '.2%')))
df_fmy_group_new.loc[:, '进件出库率'] = (df_fmy_group_new.出库/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '取消比例'] = (df_fmy_group_new.客户取消/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '人审转化率'] = (df_fmy_group_new.出库/df_fmy_group_new.机审通过件).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '人审拒绝率'] = (df_fmy_group_new.人审拒绝/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
# df_fmy_group_new.set_index('下单日期')

# 取消拒绝原因

In [ ]:
# path_result = 'F:/myfile/数据提取及分析/'
# excel_name = f'取消&拒绝原因_{t_date}.xlsx'
# dfms = df2
# # 取消原因 总体
# qx_zt_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

# # 拒绝原因总体
# jj_zt_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# #免审进件的电审拒绝原因
# dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
# jj_ms_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# #免审进件的取消原因
# dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
# qx_ms_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

# with pd.ExcelWriter(path_result + excel_name, engine='xlsxwriter') as writer:  
#     qx_zt_df.to_excel(writer, sheet_name='取消原因-总体', startrow=0, index=True)
#     jj_zt_df.to_excel(writer, sheet_name='拒绝原因-总体', startrow=0, index=True) 
#     jj_ms_df.to_excel(writer, sheet_name='拒绝原因-免审', startrow=0, index=True) 
#     qx_ms_df.to_excel(writer, sheet_name='取消原因-免审', startrow=0, index=True)

In [ ]:
# df_dsjj = df2[df2['decision_result'].str.contains(pat="电审拒绝",regex=False)==True]


In [ ]:
# from datetime import datetime, timedelta
# # 获取昨日时间并转化为字符串
# from datetime import datetime  

# df2['取消原因2'] = df2['cancel_reason'].str.split('：')
# df2['取消原因2'] = df2['取消原因2'].apply(lambda x: x[-1] if x is not None else x)
# y_date = datetime.today() - timedelta(days=1)

# y_date = y_date.strftime('%Y-%m-%d')
# df2_new = df2[(df2['下单日期'] == y_date)&(df2['拒绝理由'].notna())]
# # 切割拒绝理由内容，取第一个逗号前的内容
# # df2_new['拒绝理由'] = df2_new['拒绝理由'].str.split(',').str[0]
# df_quju = df2_new
# # 取消原因_总体
# qx_zt_df = pd.crosstab(df_quju["下单日期"],df_quju["取消原因2"],margins=False)
# # 拒绝理由_总体
# jj_zt_df = pd.crosstab(df_quju["下单日期"],df_quju["拒绝理由"],margins=False)

# # jj_zt_df[jj_zt_df['下单日期'] == '2025-07-15']

# qx_zt_df
# df_quju.groupby('取消原因2').agg({'order_id': 'count'}).sort_values(by='order_id', ascending=False).rename(columns={'order_id': '订单数'})


In [ ]:
# jj_zt_df.T

# 商家转化数据

In [ ]:
df_contain["待审核"] = np.where(df_contain["审核状态"]=='待审核',1,0)
df_contain["前置拦截"] = np.where(df_contain["审核状态"]=='前置拦截',1,0)
df_contain["机审强拒"] = np.where(df_contain["审核状态"]=='机审强拒',1,0)
df_contain["人审拒绝"] = np.where(df_contain["审核状态"]=='人审拒绝',1,0)
df_contain["客户取消"] = np.where(df_contain["审核状态"]=='客户取消',1,0)
df_contain["出库前风控强拒"] = np.where(df_contain["审核状态"]=='出库前风控强拒',1,0)
df_contain["无法联系"] = np.where(df_contain["审核状态"]=='无法联系',1,0)
df_contain["是否进件"] = np.where(df_contain["进件"]=='进件',1,0)
df_contain["是否出库"] = np.where(df_contain["status"].isin([2,3,4,5,6,8,15]),1,0)

df_contain["进件前取消"] = np.where(df_contain["审核状态"]=='进件前取消',1,0)

## 澄心优租

In [ ]:
# 澄心优租
cxyz = all_models.merchant_names(df_contain, '澄心优租', '下单日期')
cxyz = cxyz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]]
cxyz = clean.continuous_dates(cxyz)
cxyz_new = cxyz.reset_index(names=['下单日期'])
# cxyz_new

## 优优2店

In [ ]:
# 优优2店
# yy2 = all_models.merchant_names(df_contain,'优优2店', '下单日期')
# yy2 = yy2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# yy2


## 北京海鸟窝科技有限公司

In [ ]:
# 北京海鸟窝科技有限公司
hnw = all_models.merchant_names(df_contain,'北京海鸟窝科技有限公司', '下单日期')
hnw = hnw[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
hnw = clean.continuous_dates(hnw)
hnw_new = hnw.reset_index(names=['下单日期'])


## 租着用电脑数码

In [ ]:
# 租着用电脑数码
zzy = all_models.merchant_names(df_contain,'租着用电脑数码', '下单日期')
zzy = zzy[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
zzy = clean.continuous_dates(zzy)
zzy_new = zzy.reset_index(names=['下单日期'])

## 格木木二奢名品

In [ ]:
# 格木木二奢名品
gmm = all_models.merchant_names(df_contain,'格木木二奢名品', '下单日期')
gmm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


## 广州康基贸易有限公司

In [ ]:
# 广州康基贸易有限公司
kjmy = all_models.merchant_names(df_contain,'广州康基贸易有限公司', '下单日期')
kjmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


## 趣智数码

In [ ]:
# 趣智数码
qzsm = all_models.merchant_names(df_contain,'趣智数码', '下单日期')
qzsm = qzsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
qzsm = clean.continuous_dates(qzsm)
qzsm_new = qzsm.reset_index(names=['下单日期'])


## 汇客好租

In [ ]:
# 汇客好租
hkhz = all_models.merchant_names(df_contain,'汇客好租', '下单日期')
hkhz = hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
hkhz = clean.continuous_dates(hkhz)
hkhz_new = hkhz.reset_index(names=['下单日期'])
# 电脑
df_contain_dy = df_contain[(df_contain.归属渠道=='抖音渠道')&(df_contain.classify_id.isin([125,126,127,185]))]
hkhz_dy = all_models.merchant_names(df_contain_dy,'汇客好租', '下单日期')
hkhz_dy = hkhz_dy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
hkhz_dy = clean.continuous_dates(hkhz_dy)
hkhz_dy_new = hkhz_dy.reset_index(names=['下单日期'])

## 小蚂蚁租机

In [ ]:
# 小蚂蚁租机
xmy = all_models.merchant_names(df_contain,'小蚂蚁租机', '下单日期')
xmy = xmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
if len(xmy)!=0:
    xmy = clean.continuous_dates(xmy)
xmy_new = xmy.reset_index(names=['下单日期'])


## 乙辉数码

In [ ]:
# 乙辉数码
yhsm = all_models.merchant_names(df_contain,'乙辉数码', '下单日期')
yhsm = yhsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# yhsm = clean.continuous_dates(yhsm)
yhsm_new = yhsm.reset_index()
# yhsm_new

## 兴鑫兴通讯

In [ ]:
# 兴鑫兴通讯
xxx = all_models.merchant_names(df_contain,'兴鑫兴通讯', '下单日期')
xxx = xxx[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xxx_new = xxx.reset_index()
# xxx_new

## 呱子笔记本电脑

In [ ]:
# 呱子笔记本电脑
# gz = all_models.merchant_names(df_contain,'呱子笔记本电脑', '下单日期')
# gz = gz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# gz = clean.continuous_dates(gz)
# gz_new = gz.reset_index(names=['下单日期'])
# gz_new.set_index('下单日期')

## 南京聚格网络科技

In [ ]:
jgwl = all_models.merchant_names(df_contain,'南京聚格网络科技', '下单日期')
jgwl = jgwl[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
try:
    jgwl = clean.continuous_dates(jgwl)
except:
    print(KeyError)
jgwl_new = jgwl.reset_index(names=['下单日期'])
# jgwl_new.set_index('下单日期')

## 人人享租

In [ ]:
rrxz = all_models.merchant_names(df_contain,'人人享租', '下单日期')
rrxz = rrxz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# rrxz = clean.continuous_dates(rrxz)
rrxz_new = rrxz.reset_index(names=['下单日期'])
# rrxz_new

## 崇胜数码

In [ ]:
# cssm = all_models.merchant_names(df_contain,'崇胜数码', '下单日期')
# cssm = cssm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# cssm_new = cssm.reset_index(names=['下单日期'])

## 喜卓灵租机

In [ ]:

xzl = all_models.merchant_names(df_contain,'喜卓灵租机', '下单日期')
xzl = xzl[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xzl = clean.continuous_dates(xzl)
xzl_new = xzl.reset_index(names=['下单日期'])
# xzl_new

## 喜卓灵新租机

In [ ]:
xzl_news = all_models.merchant_names(df_contain,'喜卓灵新租机', '下单日期')
xzl_news = xzl_news[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# xzl_new = clean.continuous_dates(xzl_new)
xzl_news = xzl_news.reset_index(names=['下单日期'])
# xzl_news

# 每日各渠道订单统计 导出

In [ ]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date
df['下单日期']

In [ ]:
path = r'\\digua\迪瓜租机\20.日报数据相关输出/'
# 每日各渠道去重订单统计结果表格导出
# df = df[df.归属渠道=='搜索渠道']
gsqd_qcdd = pd.crosstab(df["下单日期"],df["归属渠道"],margins=True)
lyqd_qcdd = pd.crosstab(df["下单日期"],df["来源渠道"],margins=True)
hdmc_qcdd = pd.crosstab(df["下单日期"],df["activity_name"],margins=True)
#每日各渠道进件统计结果表格导出
# df_j = df_j[df_j.归属渠道=='搜索渠道']
gsqd_jj = pd.crosstab(df_j["下单日期"],df_j["归属渠道"],margins=True)
lyqd_jj = pd.crosstab(df_j["下单日期"],df_j["来源渠道"],margins=True)
hdmc_jj = pd.crosstab(df_j["下单日期"],df_j["activity_name"],margins=True)
# 各归属渠道 出库 导出
# dfck = dfck[dfck.归属渠道=='搜索渠道']
gsqd_ck = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
lyqd_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
hdmc_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 搜索-租手机、搜索-租苹果手机、搜索下挂-商品、搜索直达banner、搜索下挂券、顶部搜索框的搜索结果页
df_j_new = df_j[df_j.来源渠道.isin(['搜索渠道','搜索-租手机','搜索-租苹果手机','搜索下挂-商品','搜索直达banner','搜索下挂券','顶部搜索框的搜索结果页'])]
dfck_new = dfck[dfck.来源渠道.isin(['搜索渠道','搜索-租手机','搜索-租苹果手机','搜索下挂-商品','搜索直达banner','搜索下挂券','顶部搜索框的搜索结果页'])]
# 搜索渠道进件数据统计结果表格导出，交叉表行索引为下单日期和商品ID_机型，列索引为来源渠道
ss_jj = pd.crosstab(index=[df_j_new["下单日期"],df_j_new["商品ID_机型"]],columns=df_j_new["来源渠道"],margins=True)
# 搜索渠道出库数据统计结果表格导出，交叉表行索引为下单日期和商品ID_机型，列索引为来源渠道
ss_ck = pd.crosstab(index=[dfck_new["下单日期"],dfck_new["商品ID_机型"]],columns=dfck_new["来源渠道"],margins=True)
try:
    with pd.ExcelWriter(path + f'日报相关数据输出_{t_date}.xlsx', engine='xlsxwriter') as writer:
        # 去重订单
        gsqd_qcdd.to_excel(writer, sheet_name='归属渠道去重订单统计', index=True)  
        lyqd_qcdd.to_excel(writer, sheet_name='来源渠道去重订单统计', index=True)
        hdmc_qcdd.to_excel(writer, sheet_name='活动名称去重订单统计', index=True)
        # 进件
        gsqd_jj.to_excel(writer, sheet_name='归属渠道进件统计', index=True)
        lyqd_jj.to_excel(writer, sheet_name='来源渠道进件统计', index=True)
        hdmc_jj.to_excel(writer, sheet_name='活动名称进件统计', index=True)
        # 出库
        gsqd_ck.to_excel(writer, sheet_name='归属渠道出库统计', index=True)
        lyqd_ck.to_excel(writer, sheet_name='来源渠道出库统计', index=True)
        hdmc_ck.to_excel(writer, sheet_name='活动名称出库统计', index=True)
        ss_jj.to_excel(writer, sheet_name='搜索渠道进件统计', index=True)  
        ss_ck.to_excel(writer, sheet_name='搜索渠道出库统计', index=True)
except PermissionError:
    print(PermissionError)



# 来源渠道统计进件出库数据

In [ ]:


# path="D:/需求/"
# # 搜索-租手机、搜索-租苹果手机、搜索下挂-商品、搜索直达banner、搜索下挂券、顶部搜索框的搜索结果页
# df_j_new = df_j[df_j.来源渠道.isin(['搜索渠道','搜索-租手机','搜索-租苹果手机','搜索下挂-商品','搜索直达banner','搜索下挂券','顶部搜索框的搜索结果页'])]
# dfck_new = dfck[dfck.来源渠道.isin(['搜索渠道','搜索-租手机','搜索-租苹果手机','搜索下挂-商品','搜索直达banner','搜索下挂券','顶部搜索框的搜索结果页'])]
# # 搜索渠道进件数据统计结果表格导出，交叉表行索引为下单日期和商品ID_机型，列索引为来源渠道
# ss_jj = pd.crosstab(index=[df_j_new["下单日期"],df_j_new["商品ID_机型"]],columns=df_j_new["来源渠道"],margins=True)
# # 搜索渠道出库数据统计结果表格导出，交叉表行索引为下单日期和商品ID_机型，列索引为来源渠道
# ss_ck = pd.crosstab(index=[dfck_new["下单日期"],dfck_new["商品ID_机型"]],columns=dfck_new["来源渠道"],margins=True)
# # 导出
# try:
#     with pd.ExcelWriter(path + f'搜索渠道进件出库数据统计_{t_date}.xlsx', engine='xlsxwriter') as writer:
#         ss_jj.to_excel(writer, sheet_name='搜索渠道进件统计', index=True)  
#         ss_ck.to_excel(writer, sheet_name='搜索渠道出库统计', index=True)
# except PermissionError:
#     print(PermissionError)


# 总体 "租完即送"占比

In [ ]:
# 获取拒量的出库和非拒量的出库
dfck_fjl = dfck[~dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
dfck_jl = dfck[dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]

In [ ]:
#总体 "租完即送"占比（拒量）
df_rent_jlck = pd.crosstab(dfck_jl["下单日期"],dfck_jl["租赁方案"],margins=True)
# 填充不连续日期的数据
df_rent_jlck = clean.continuous_dates(df_rent_jlck.iloc[:-1, :])
df_rent_jlck = df_rent_jlck.reset_index(names=['下单日期'])
df_rent_jlck['下单日期'] = pd.to_datetime(df_rent_jlck['下单日期'])
df_rent_jlck = df_rent_jlck.set_index('下单日期')
df_rent_jlck.rename(columns={'租完即送': '租完即送_拒量', '租完归还': '租完归还_拒量', 'All': 'All_拒量'}, inplace=True)
df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
try:
    df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
except:
    df_rent_jlck.loc[:, '租完归还_拒量'] = 0
    df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))


In [ ]:
#总体 "租完即送"占比（排除拒量）
df_rent  = pd.crosstab(df_j["下单日期"],df_j["租赁方案"],margins=True)
df_rent=df_rent.rename(columns={"租完即送":"租完即送进件","租完归还":"租完归还进件","All":"合计进件",})
df_rent_ck=pd.crosstab(dfck_fjl["下单日期"],dfck_fjl["租赁方案"],margins=True)
df_rent_ck=df_rent_ck.rename(columns={"租完即送":"租完即送出库","租完归还":"租完归还出库","All":"合计出库",})
df_r=pd.merge(df_rent,df_rent_ck,left_index=True, right_index=True)
df_r["租完即送进件占比"]=df_r["租完即送进件"]/df_r["合计进件"]
df_r["租完即送进件占比"]=df_r["租完即送进件占比"].apply(lambda x:format(x,".2%"))
df_r["租完归还进件占比"]=df_r["租完归还进件"]/df_r["合计进件"]
df_r["租完归还进件占比"]=df_r["租完归还进件占比"].apply(lambda x:format(x,".2%"))
df_r["租完即送出库占比"]=df_r["租完即送出库"]/df_r["合计出库"]
df_r["租完即送出库占比"]=df_r["租完即送出库占比"].apply(lambda x:format(x,".2%"))
df_r["租完归还出库占比"]=df_r["租完归还出库"]/df_r["合计出库"]
df_r["租完归还出库占比"]=df_r["租完归还出库占比"].apply(lambda x:format(x,".2%"))
df_r["租完即送转化率"]=df_r["租完即送出库"]/df_r["租完即送进件"]
df_r["租完即送转化率"]=df_r["租完即送转化率"].apply(lambda x:format(x,".2%"))
df_r["租完归还转化率"]=df_r["租完归还出库"]/df_r["租完归还进件"]
df_r["租完归还转化率"]=df_r["租完归还转化率"].apply(lambda x:format(x,".2%"))
df_r = df_r.iloc[:-1, :]
df_r = df_r.reset_index()
df_r.loc[:, '下单日期'] = pd.to_datetime(df_r['下单日期'])
df_r.set_index('下单日期', inplace=True)
# 拼接拒量出库
df_r1 = df_r.merge(df_rent_jlck, on='下单日期', how='left')

df_r1 = df_r1[["租完即送进件","租完归还进件","租完归还进件占比","租完即送进件占比","租完即送出库","租完归还出库", '租完即送_拒量', '租完归还_拒量', '租完即归还库占比_拒量', '租完即送出库占比_拒量', "租完归还出库占比","租完即送出库占比", '租完即送转化率','租完归还转化率']]#.iloc[:-1, :]
df_r1 = clean.continuous_dates(df_r1)
df_r_new = df_r1.reset_index(names=['下单日期'])
# df_r_new

# 总体出库订单碎屏险购买数据

## 总体

In [ ]:
def ck(dfck, name):
    # '服务订单状态：1、待支付；2、待确认；3、服务中；4、已失效；5、已取消；6、已退款',
    dfck[f'已取消'] = np.where(dfck["service_status"]==5,1,0)
    dfck[f'已退款'] = np.where(dfck["service_status"]==6,1,0)
    dfck[f'待支付'] = np.where(dfck["service_status"]==1,1,0)
    dfck[f'待确认'] = np.where(dfck["service_status"]==2,1,0)
    dfck[f'服务中'] = np.where(dfck["service_status"]==3,1,0)

    df_s3  =  dfck.groupby(["下单日期"]).agg({f'已取消':'sum',f'已退款':'sum', f'待支付':'sum', f'待确认':'sum',f'服务中':'sum'})
    
    df_s3[f"最终实际支付碎屏险出库_{name}"]=df_s3[f"待确认"]+df_s3[f"服务中"] +df_s3[f"待支付"]
    return df_s3

In [ ]:
# 不包含京享租
dfck_fjl_fjd = dfck_fjl[dfck_fjl.归属渠道!='京东渠道']
dfck_jl_fjd = dfck_jl[dfck_jl.归属渠道!='京东渠道']
#总体出库订单碎屏险购买数据
df_bx_ck=pd.crosstab(dfck_fjl_fjd["下单日期"],dfck_fjl_fjd["buy_service_product"],margins=True)
df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
#总体出库订单碎屏险购买数据——拒量
df_bx_ck_jl=pd.crosstab(dfck_jl_fjd["下单日期"],dfck_jl_fjd["buy_service_product"],margins=True)
df_bx_ck_jl=df_bx_ck_jl.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库_拒量",})

# 非拒量
df_s3 = ck(dfck_fjl_fjd, 'All')
# 拒量
df_s3_jl = ck(dfck_jl_fjd, '拒量')

# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.drop(columns={"All"})
df_s2merge=pd.merge(df_bx_ck,df_s3,left_index=True, right_index=True)
df_s3merge=pd.merge(df_bx_ck_jl,df_s3_jl,left_index=True, right_index=True)
df_s2merge = df_s2merge.merge(df_s3merge[['合计出库_拒量', '最终实际支付碎屏险出库_拒量']], left_index=True, right_index=True, how='left')
df_s2merge["实际支付碎屏险比例"]=df_s2merge["最终实际支付碎屏险出库_All"]/df_s2merge["合计出库"]
df_s2merge["实际支付碎屏险比例"]=df_s2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))
df_s2merge.loc[:, '实际支付碎屏险比例_拒量'] = (df_s2merge.最终实际支付碎屏险出库_拒量/df_s2merge.合计出库_拒量).map(lambda x: format(x, '.2%'))

df_s2merge = df_s2merge[['不购买碎屏险出库','购买碎屏险出库','合计出库', '合计出库_拒量','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库_拒量','最终实际支付碎屏险出库_All', '实际支付碎屏险比例_拒量',  '实际支付碎屏险比例']]
df_s2merge = clean.continuous_dates(df_s2merge)
df_s2merge = df_s2merge.reset_index(names=['下单日期'])
# df_s2merge

In [ ]:
#芝麻租物出库订单碎屏险购买数据
dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
df_zw3=pd.crosstab(dfckzw["下单日期"],dfckzw["buy_service_product"],margins=True)
df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})

dfckzw['已取消'] = np.where(dfckzw["service_status"]==5,1,0)
dfckzw['已退款'] = np.where(dfckzw["service_status"]==6,1,0)
dfckzw['待支付'] = np.where(dfckzw["service_status"]==1,1,0)
dfckzw['待确认'] = np.where(dfckzw["service_status"]==2,1,0)
dfckzw['服务中'] = np.where(dfckzw["service_status"]==3,1,0)

df_zws3  =   dfckzw.groupby(["下单日期"]).agg({'已取消':'sum','已退款':'sum', '待支付':'sum', '待确认':'sum','服务中':'sum'})  
# =pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})

df_zws2merge = pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
df_zws2merge["最终实际支付碎屏险出库"] = df_zws2merge["待确认"]+df_zws2merge["服务中"]+df_zws2merge["待支付"]
df_zws2merge["实际支付碎屏险比例"] = df_zws2merge["最终实际支付碎屏险出库"]/df_zws2merge["合计出库"]
df_zws2merge["实际支付碎屏险比例"] = df_zws2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))


df_zws2merge = df_zws2merge[['不购买碎屏险出库','购买碎屏险出库','合计出库','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库','实际支付碎屏险比例']]
df_zws2merge = clean.continuous_dates(df_zws2merge)
df_zws2merge_new = df_zws2merge.reset_index(names=['下单日期1'])
# df_zws2merge_new = df_zws2merge_new.rename(columns={'下单日期': '下单日期1'})
# df_zws2merge_new

## 整合

In [ ]:
df_s2merge_new = df_s2merge#.reset_index()
df_s2merge_new.loc[:, '碎屏险实际出库占比'] = (df_s2merge_new.最终实际支付碎屏险出库_All/df_s2merge_new.购买碎屏险出库).map(lambda x: format(x,".2%"))
df_s2merge_new.loc[:, '非芝麻实际支付碎屏险比例'] = ((df_s2merge_new.购买碎屏险出库-df_zws2merge_new.购买碎屏险出库)/(df_s2merge_new.合计出库-df_zws2merge_new.合计出库)).map(lambda x: format(x,".2%"))
df_s2merge_all_new = pd.concat([df_s2merge_new, df_zws2merge_new], axis=1)
# df_s2merge_all_new.set_index('下单日期')

## 京享租

In [ ]:
df_jd2_ck = df_jd2[df_jd2.是否出库==1]
df_jd2_sp=pd.crosstab(df_jd2_ck["下单日期"],df_jd2_ck["buy_service_product"],margins=True)
df_jd2_sp=df_jd2_sp.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
df_jdsp = ck(df_jd2_ck, 'All')
df_jd_merge=pd.merge(df_jd2_sp,df_jdsp,left_index=True, right_index=True)
df_jd_merge["实际支付碎屏险比例"]=(df_jd_merge["最终实际支付碎屏险出库_All"]/df_jd_merge["合计出库"]).map(lambda x: format(x, '.2%'))
df_jd_merge = df_jd_merge.reset_index()
df_jd_merge.loc[:, '下单日期'] = df_jd_merge.下单日期.astype('object')
df_jd_group_new = df_jd_group_new.merge(df_jd_merge[['下单日期', '合计出库', '最终实际支付碎屏险出库_All', '实际支付碎屏险比例']], on='下单日期', how='left')


# 拒量

In [ ]:
# # 设置时间段
# now_day = datetime.now()
# seven_ago = now_day - pd.Timedelta(days=8)
# one_ago = now_day - pd.Timedelta(days=1)
# # 获取特定时间段的拒量订单的转化数据和明细
# # df_jl = df[(df.下单日期>=seven_ago)&(df.下单日期<=one_ago)&(df.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True)]
# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# # 进件数，出库数，出库率，风险等级
# df_jl_new_group = df_jl_new.groupby('下单日期').agg({'是否进件': 'sum', '是否出库': 'sum'})
# df_jl_new_group.rename(columns={'是否进件': '进件数', '是否出库': '出库'}, inplace=True)
# df_jl_new_group['进件出库率'] = (df_jl_new_group.出库/df_jl_new_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_new2 = df_jl_new[['下单日期', 'order_number', '策略命中等级']]
# df_jl_new2_group = df_jl_new2.groupby('策略命中等级').agg(数量=('order_number', 'count'))

# # 获取当前是星期几
# # Monday = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
# # if Monday == 'Monday':
# #     with pd.ExcelWriter(f'F:/需求/廖志彧需求/拒量数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
# #         df_jl_new_group.to_excel(writer, sheet_name='转化数据')
# #         df_jl_new2_group.to_excel(writer, sheet_name='拒量数据明细')
#             # df_jl_new['order_number'].to_excel(writer, sheet_name='拒量明细', index=False)


# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='left')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]
# # with pd.ExcelWriter(f'F:/需求/廖志彧需求/分配数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
# #     df_jl_name_new.to_excel(writer, sheet_name='出库单分配人', index=False)


# # 首逾
# date1 = '2025-05'
# date2 = '2025-06'
# df_new = df_jl[df_jl.下单月份==date1]
# df_new = df_new[df_new.是否出库==1]
# df_stages_new = df_stages[df_stages['order_id'].notna()]
# df_stages_new['order_id'] = df_stages_new['order_id'].astype(int)
# df_stages_new.loc[:, '预计还款月份'] = df_stages_new.refund_date.dt.strftime('%Y-%m')
# df_new_merge = df_new.merge(df_stages_new, on='order_id', how='left')
# # df_jl_stages_new = df_new_merge[(df_new_merge.预计还款月份==date2)&(df_new_merge.sort==2)]
# df_jl_stages_new = df_new_merge[(df_new_merge.sort==2)]
# df_jl_stages_new.loc[:, 'now_day'] = pd.to_datetime(now_day.strftime('%Y-%m-%d'))
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.refund_date<=df_jl_stages_new.now_day]
# df_jl_stages_new.loc[:, 'overdue_day'] = np.where(df_jl_stages_new.reality_refund_date.notna(), 
#                                                 np.where(df_jl_stages_new.reality_refund_date>df_jl_stages_new.refund_date, (df_jl_stages_new.reality_refund_date-df_jl_stages_new.refund_date), 0)
#                                                 , np.where(df_jl_stages_new.refund_date>df_jl_stages_new.now_day, 0, (df_jl_stages_new.now_day-df_jl_stages_new.refund_date)))
# df_jl_stages_new.loc[:, 'overdue_day'] = df_jl_stages_new['overdue_day'].apply(lambda x: x.days)
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.overdue_day>0]

# # df_jl_stages_news = df_jl_stages_new[df_jl_stages_new.overdue_day>0][['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_new[['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day', 'status2']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_news.merge(df_name_new[['order_id', '分配人']], left_on='订单id', right_on='order_id', how='left')
# df_jl_stages_news.loc[:, '是否逾期'] = np.where((df_jl_stages_news.status2=='租赁中')&(df_jl_stages_news.实还日期.isna())&(df_jl_stages_news.逾期天数>0), 1, 0)
# df_jl_stages_news_g = df_jl_stages_news.groupby('分配人').agg({'order_id': 'count', '是否逾期': 'sum'}).rename(columns={'order_id': '出库'})
# df_jl_stages_news_g.loc[:, '逾期/出库'] = (df_jl_stages_news_g.是否逾期/df_jl_stages_news_g.出库).map(lambda x: format(x, '.2%'))
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/首逾_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jl_stages_news.to_excel(writer, sheet_name='出库订单', index=False)
#     df_jl_stages_news_g.to_excel(writer, sheet_name='首逾订单')

In [ ]:
# df_jl_new[['下单日期', 'order_number', '策略命中等级', '是否出库']].to_excel(f'F:/需求/总需求/拒量回捞数据_{t_date}.xlsx', index=False)

## 拒量日报

In [ ]:
df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
df_jl_group = df_jl.groupby('下单日期').agg({'是否进件': 'sum', '人审拒绝': 'sum', '风控通过件': 'sum', '客户取消': 'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum', '已退款': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库': '出库'})
df_jl_group.loc[:, '客户取消'] = df_jl_group.客户取消+df_jl_group.已退款
df_jl_group.loc[:, '风控通过率'] = (df_jl_group.风控通过件/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '进件出库率'] = (df_jl_group.出库/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '取消比例'] = (df_jl_group.客户取消/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '电审拒绝率'] = (df_jl_group.人审拒绝/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '出库前强拒比例'] = (df_jl_group.出库前风控强拒/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '无法联系占比'] = (df_jl_group.无法联系/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group_new = df_jl_group[['进件数', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '取消比例', '电审拒绝率', '出库前强拒比例', '无法联系占比']].reset_index()

# df_jl_group_new

# 挽留

In [ ]:
df_wl_merge = df_wl.merge(df[['order_number', '是否出库']], on='order_number', how='left')
df_wl_merge.loc[:, '下单日期'] = pd.to_datetime(df_wl_merge.下单日期)
df_wl_merge.loc[:, '挽留成功'] = np.where(df_wl_merge.task_status=='SUCCESS', 1, 0)
df_wl_g = df_wl_merge.groupby('下单日期').agg({'order_number': 'nunique', '挽留成功': 'sum', '是否出库': 'sum'}).rename(columns={'order_number': '挽留数', '是否出库': '出库'})
df_wl_g_merge = df_all[['去重订单数', '进件数', '客户取消']].merge(df_wl_g, on='下单日期', how='left')
df_wl_g_merge.loc[:, '进件出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.进件数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '订单出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.去重订单数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '挽留成功率'] = (df_wl_g_merge.挽留成功/df_wl_g_merge.挽留数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '挽留出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.挽留数).map(lambda x: format(x, '.2%'))
df_wl_g_merge = clean.continuous_dates(df_wl_g_merge)
df_wl_g_merge_new = df_wl_g_merge.reset_index(names=['下单日期'])
# df_wl_g_merge_new

# 获取日报文件名称并修改

In [ ]:
now = datetime.now()
now_day = pd.to_datetime(now.strftime("%Y-%m-%d"))
now_date = now.strftime('%m%d')
before_date = (now-timedelta(days=1)).strftime('%m%d')
before_date_two = (now-timedelta(days=2)).strftime('%m%d')
before_date_four = (now-timedelta(days=4)).strftime('%m%d')
monday = pd.to_datetime(now.strftime('%Y-%m-%d')).day_name()

In [ ]:
# 指定目录路径
directory = 'D:/excel_data'

# 查找所有 .xlsx 和 .xls 文件
excel_files = glob.glob(os.path.join(directory, '*.xlsx')) + glob.glob(os.path.join(directory, '*.xls'))

for old_file_path in excel_files:
    # 提取文件名（不带路径）
    old_filename = os.path.basename(old_file_path)
    old_date = old_filename.split('.')[0][-4:]
    new_filename = old_filename.replace(old_date, before_date)
    # 将新的文件名添加到路径
    new_file_path = os.path.join(directory, new_filename)

    # 确保新文件名不存在，以避免覆盖文件
    if not os.path.exists(new_file_path):
        os.rename(old_file_path, new_file_path)
        print(f"文件 {old_filename} 已重命名为: {new_filename}")
    else:
        print(f"跳过文件 {old_filename}，因为新文件名已存在。")

# 读取日报

## 获取日期及星期

In [ ]:

# 中文星期映射 获取星期几

# 设置需要打开的文件和密码
# file_path1 = f'F:/日报/日报/迪瓜租机订单日报截止{before_date}.xlsx'
file_path1 = f'D:/excel_data/new迪瓜租机订单日报截止{before_date}.xlsx'

# file_path2 = f'F:/日报/日报/商户订单统计表截至{before_date}.xlsx'
file_path2 = f'D:/excel_data/new商户订单统计表截至{before_date}.xlsx'
password = '20240101'

## 日报数据

In [ ]:
def Open_Excel2(df_list, path, password, col_list, key_list, key):
    '''
    对日报数据进行自动更新
    :param df: 日报各个渠道的df
    :param path: 日报的excel文件路径
    :param password: excel文件的密码
    :param sheet_name: excel的sheet页名称
    :param col: 列的位置，如A列，B列等
    :param col_len: 每个模块之间的间隔宽度
    :param key: 判断是否有进行合计
    :return:
    '''
    # 定义文件路径和密码
    file_path = path
    password = password
    # 使用 xlwings 打开加密的 Excel 文件
    app = xw.App(visible=False)
    # 获取今天是星期几
    week = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
    # 初始化数据列长
    col_len = 1
    try:
        wb = xw.Book(file_path, password=password)
        # 获取工作表
        sheet_names = wb.sheet_names
        num = 1 if key=='日报' else 0
        for idx, df_lis in enumerate(df_list):
            sheet = wb.sheets[sheet_names[idx+num]]
            # print(sheet_names[idx])
            col = col_list[idx]
            key = key_list[idx]
            for idx1, df in enumerate(df_lis):
                # 如果df没有数据则不执行操作
                # print(len(df))
                if len(df)!=0:
                    # 获取该工作表的最后一行，并插入一行
                    # col_len:判断是否是第一列第一个需要插入的数据；
                    # key：值等于1用来判断是否是免人审数据，免审订单转化，押金类型这三个数据页在最后一行的前三行插入； 值不等于1则在最后一行插入并且起始行数最后减4
                    # 获取最后一行的日期数据进行判断
                    last_row = sheet.range(col + str(sheet.cells.last_cell.row)).end('up').row
                    # print(last_row)
                    last_date = sheet.range(f'{col}{last_row}').value if key==0 else sheet.range(f'{col}{last_row - 4}').value
                    # 获取前一天的日期
                    before_date = pd.to_datetime((datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'))
                    if col_len == 1:
                        if before_date != last_date:
                            # print(f'before_date: {before_date},last_date:{last_date}')
                            diff = (before_date - last_date).days
                            if key==0:
                                # 判断最后一行日期是不是前一天的日期，如不是则差几天就插入几行，如是则不插入新的行
                                sheet.api.Rows(last_row).Insert() if diff == 1 else sheet.api.Rows(f'{last_row}:{last_row + diff - 1}').Insert()
                            else:
                                sheet.api.Rows(last_row - 3).Insert() if diff == 1 else sheet.api.Rows(f'{last_row - diff + 1}:{last_row}').Insert()

                    # 重新获取最后一行
                    last_row = sheet.used_range.last_cell.row
                    # 将 DataFrame 写入固定的位置
                    # 对应Excel中的行和列，起始行数
                    start_row = last_row - len(df) - 1 if key==0 else last_row - len(df) - 3
                    # print(start_row)
                    # 定位写入区域左上角单元格（行, 列）
                    target_range = sheet.range((start_row, col_len))
                    # 一次性写入整个DataFrame数据
                    target_range.options(index=False, header=False).value = df
                    # 获取上一个df的宽度并相加
                    col_len += 1 + len(df.columns)
            # 新的sheet页，数据列长重新赋值
            col_len = 1
        
        # 保存并重新加密文件
        wb.save(password=password)
    finally:
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()


In [ ]:
# 日报数据
df_list = [[df_all_new[:-1], df_ss_group_new[:-1], df_dr_group_new[:-1], df_zm_group_new[:-1], df_zw_group_new[:-1], df_tc[:-1], df_dy_group_new[:-1], df_zfb_group1[:-1]], 
            [df_ms_new3[:-1]], [df_ms_o_group_new[:-1]],[df_qmy_group_new[:-1], df_fmy_group_new[:-1]], [df_r_new[:-1]], [df_s2merge_all_new[:-1]], [df_jl_group_new[:-1]], 
            [df_wl_g_merge_new[:-1]], [df_jd_group_new[:-1], df_jdsc_group_new[:-1]]]
col_list = ['C', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A']
key_list = [0, 1, 1, 1, 0, 0, 0, 0, 0]
Open_Excel2(df_list, file_path1, password, col_list, key_list, key='日报')
# 商家数据
sj_df_list = [[cxyz_new[:-1]],[hnw_new[:-1]],[zzy_new[:-1]],[qzsm_new[:-1]],[hkhz_new[:-1], hkhz_dy_new[:-1]],[xmy_new[:-1]],[rrxz_new[:-1]],[jgwl_new[:-1]],[xzl_new[:-1]],[xzl_news[:-1]]]
sj_col_list = ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
sj_key_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Open_Excel2(sj_df_list, file_path2, password, sj_col_list, sj_key_list, key='商家')


In [ ]:
# sheet_name = '2025年'
# # 自动写入日报数据 总体

# col_len = all_models.Open_Excel2(df_all_new[:-1], file_path1, password, sheet_name, 'C')
# # 搜索渠道
# col_len1 = all_models.Open_Excel2(df_ss_group_new[:-1], file_path1, password, sheet_name, 'C', col_len)
# # 单人会话
# df_dr_group_new = df_dr_group.reset_index()
# col_len2 = all_models.Open_Excel2(df_dr_group_new[:-1], file_path1, password, sheet_name, 'C', col_len1)
# # 芝麻租物
# col_len3 = all_models.Open_Excel2(df_zm_group_new[:-1], file_path1, password, sheet_name, 'C', col_len2)
# # 纯租物
# df_zw_group_new = df_zw_group.reset_index()
# col_len4 = all_models.Open_Excel2(df_zw_group_new[:-1], file_path1, password, sheet_name, 'C',  col_len3)
# # 总体剔除直播
# df_tc_new = df_tc.reset_index()
# col_len5 = all_models.Open_Excel2(df_tc_new[:-1], file_path1, password, sheet_name, 'C',  col_len4)
# # 抖音渠道
# df_dy_group_new = df_dy_group.reset_index()
# col_len6 = all_models.Open_Excel2(df_dy_group_new[:-1], file_path1, password, sheet_name, 'C', col_len5)
# #  支付宝直播
# df_zfb_group_new = df_zfb_group1.reset_index()
# col_len7 = all_models.Open_Excel2(df_zfb_group_new[:-1], file_path1, password, sheet_name, 'C',  col_len6)
# ## 免人审数据
# all_models.Open_Excel2(df_ms_new3[:-1], file_path1, password, '免人审数据', 'B',  key=1)
# ## 免审订单转化统计
# all_models.Open_Excel2(df_ms_o_group_new[:-1], file_path1, password, '免审订单转化统计', 'B',  key=1)
# # 全免押
# yj_col_len = all_models.Open_Excel2(df_qmy_group_new[:-1], file_path1, password, '押金类型', 'A',  key='押金')
# # # 非免押
# all_models.Open_Excel2(df_fmy_group_new[:-1], file_path1, password, '押金类型', 'A', col_len=yj_col_len, key='押金')
# ## 租完即送占比
# all_models.Open_Excel2(df_r_new[:-1], file_path1, password, '租完即送占比', 'A', key='租完即送')
# ## 碎屏险数据
# all_models.Open_Excel2(df_s2merge_all_new[:-1], file_path1, password, '碎屏险数据', 'A', key='碎屏险')
# ## 拒量
# all_models.Open_Excel2(df_jl_group_new[:-1], file_path1, password, '拒量', 'A', key='拒量')
# ## 挽留
# all_models.Open_Excel2(df_wl_g_merge_new[:-1], file_path1, password, '挽留', 'A', key='挽留')
# ## 京享租
# col_len = all_models.Open_Excel2(df_jd_group_new[:-1], file_path1, password, '京享租', 'A', key='京享租')
# # 京东商城
# all_models.Open_Excel2(df_jdsc_group_new[:-1], file_path1, password, '京享租', 'A', key='京享租', col_len=col_len)


## 商家数据

In [ ]:
# def merchant(df, name):
#     df_max_date = df.下单日期.max()
#     if df_max_date<(now_day-timedelta(days=1)):
#         df.loc[len(df)] = np.nan
#         # return df
#         all_models.Open_Excel2(df=df, path=file_path2, password=password, sheet_name=name, col='A', key='商家')
#     else:
#         all_models.Open_Excel2(df=df, path=file_path2, password=password, sheet_name=name, col='A', key='商家')
#         # return df

In [ ]:
# # 澄心优租
# merchant(cxyz_new[:-1], '澄心优租')
# # 北京海鸟窝科技有限公司
# merchant(hnw_new[:-1], '海鸟窝')
# # 租着用
# merchant(zzy_new[:-1], '租着用')
# # 趣智数码
# merchant(qzsm_new[:-1], '趣智数码')
# # 汇客好租
# merchant(hkhz_new[:-1], '汇客好租')
# # 小蚂蚁租机
# merchant(xmy_new[:-1], '小蚂蚁租机')
# # 乙辉数码
# # merchant(yhsm_new, '乙辉数码')
# # 兴鑫兴通讯
# # merchant(xxx_new, '兴鑫兴通讯')

# # 南京聚格网络科技
# merchant(jgwl_new[:-1], '南京聚格网络科技')
# # 呱子笔记本电脑
# # merchant(gz_new[:-1], '呱子笔记本电脑')
# # 人人享租
# merchant(rrxz_new[:-1], '人人享租')
# # 崇胜数码
# # merchant(cssm_new[:-1], '崇胜数码')
# # 喜卓灵租机
# merchant(xzl_new[:-1], '喜卓灵租机')